# Directories

In [10]:
train_dir = "D:/Study Materials/Practical Materials/Thesis Files/Thesis/data/train"
test_dir = "D:/Study Materials/Practical Materials/Thesis Files/Thesis/data/train"
val_dir = "D:/Study Materials/Practical Materials/Thesis Files/Thesis/data/train"

# Imports

In [11]:
import os
import random
import matplotlib.pyplot as plt
from PIL import Image
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.python.keras.models import Model, Sequential
from tensorflow.python.keras.layers import Dense, Flatten, Dropout

In [12]:
# For VGG16
from tensorflow.python.keras.applications.vgg16 import VGG16
from tensorflow.python.keras.applications.vgg16 import preprocess_input, decode_predictions

In [13]:
# For InceptionV3
from tensorflow.python.keras.applications.inception_v3 import InceptionV3 
from tensorflow.python.keras.applications.inception_v3  import preprocess_input, decode_predictions

In [14]:
# For ResNet50
from tensorflow.python.keras.applications.resnet import ResNet50
from tensorflow.python.keras.applications.resnet import preprocess_input, decode_predictions

In [ ]:
# For Explanable AI
import lime
from lime import lime_image
from skimage.segmentation import mark_boundaries
from ipywidgets import IntProgress

# User-defined Functions

In [15]:
def transferLayer(model, layer):
    conv_model = Model(inputs=model.input, outputs=layer.output)
    # Start a new Keras Sequential model
    new_model = Sequential()

    # Add the convolutional part of the VGG16 model from above
    new_model.add(conv_model)

    # Flatten the output of the VGG16 model because it is from a convolutional layer
    new_model.add(Flatten())

    # Add a dense (aka. fully-connected) layer
    # This is for combining features that the VGG16 model has recognized in the image
    new_model.add(Dropout(0.5))

    new_model.add(Dense(512, activation="relu"))

    # Add the final layer for the actual classification
    new_model.add(Dense(2, activation="softmax"))

    return new_model


def trainableLayers(model):
    for i in range(len(list(model.layers))):
        print(list(model.layers)[i].name)
    return list(model.layers)[-1].name


def history(model, generator_train, generator_val):
    optimizer = Adam(lr=1e-5)
    loss = "categorical_crossentropy"
    metrics = ["categorical_accuracy"]
    model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
    epochs = 25
    steps_per_epoch = (generator_train.n / batch_size) / 3
    steps_val = generator_val.n / batch_size
    history = model.fit_generator(
        generator=generator_train,
        epochs=epochs,
        steps_per_epoch=steps_per_epoch,
        validation_data=generator_val,
        validation_steps=steps_val,
    )
    return history


def plotmodel(model):
    return tf.keras.utils.plot_model(
        model,
        dpi=96,
    )


def plotGraph(h):
    pd.DataFrame(h.history).plot(figsize=(8, 5))
    plt.grid(True)
    plt.gca().set_ylim(0, 1)
    return plt.show()


# Image Data Generation

In [ ]:
input_shape=224,224
datagen_train = ImageDataGenerator(rescale=1.0 / 255)
datagen_test = ImageDataGenerator(rescale=1.0 / 255)
datagen_val = ImageDataGenerator(rescale=1.0 / 255)
batch_size = 16
generator_train = datagen_train.flow_from_directory(
    directory=train_dir, target_size=input_shape, batch_size=batch_size, shuffle=True
)

generator_test = datagen_test.flow_from_directory(
    directory=test_dir, target_size=input_shape, batch_size=batch_size, shuffle=False
)

generator_val = datagen_val.flow_from_directory(
    directory=val_dir, target_size=input_shape, batch_size=batch_size, shuffle=False
)


# Explanable AI

In [ ]:
def xAI(model, img_dir):
    image = Image.open(img_dir)
    image = image.resize((224, 224))
    img = np.asarray(image)
    print(img.shape)
    explainer = lime_image.LimeImageExplainer(random_state=42)
    explanation = explainer.explain_instance(img, vgg16.predict)
    image, mask = explanation.get_image_and_mask(
        vgg16.predict(img.reshape((1, 224, 224, 3))).argmax(axis=1)[0],
        positive_only=False,
        hide_rest=False,
    )
    plt.imshow(mark_boundaries(image, mask))


# VGG16

In [ ]:
model1 = VGG16(include_top=False, input_shape=(224, 224, 3), weights="imagenet")
l1 = trainableLayers(model1)
print("Last Layer : " + l1)


In [ ]:
layer1 = model1.get_layer(l1)
vgg16 = transferLayer(model1, layer1)


In [ ]:
vgg16.summary()


In [ ]:
# For cross-validation
h1 = history(vgg16, generator_train, generator_val)
plotGraph(h1)


In [ ]:
# Using Explanable AI on VGG16
xAI(vgg16, "")


In [ ]:
xAI(vgg16, "")


In [ ]:
xAI(vgg16, "")


In [ ]:
xAI(vgg16, "")


# ResNet50

In [ ]:
model4 = ResNet50(include_top=False, input_shape=(224, 224, 3), weights="imagenet")
l4 = trainableLayers(model4)
print("Last Layer : " + l4)


In [ ]:
layer4 = model4.get_layer(l4)
r50 = transferLayer(model4, layer4)


In [ ]:
r50.summary()


In [ ]:
# For cross-validation
h4 = history(r50, generator_train, generator_val)
plotGraph(h4)


# Inception V3

In [ ]:
model3 = InceptionV3(include_top=False, input_shape=(224, 224, 3), weights="imagenet")
l3 = trainableLayers(model3)
print("Last Layer : " + l3)


In [ ]:
layer3 = model3.get_layer(l3)
iv3 = transferLayer(model3, layer3)
# plotmodel(iv3)


In [ ]:
iv3.summary()


In [ ]:
# For cross-validation
h3 = history(iv3, generator_train, generator_val)
plotGraph(h3)
